In [1]:
import numpy as np
import matplotlib.pyplot as plt
from well import Well
from geometry_keeper import GeometryKeeper
from scipy.integrate import quad
from scipy.special import k0, iti0k0
import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
outer_bound = "nnnn"
top_bound = "imp"
bottom_bound = "imp"
wtype = "frac"
n_stehf = 10
xwd = 2.5
xed = 5.
ywd = 2.5
yed = 5.
N = 20
s = 7.

params = {"nseg": N, "kx": 1., "ky": 1., "kz": 1., "omega": 1, "lambda": 1, "Fcd":100, "ref_length":1.,
         "z_ref_length": 1}
real_well = Well(xwd, ywd, outer_bound, top_bound, bottom_bound, wtype, params, n_stehf)
well = real_well.lapl_well

In [3]:
def ifb3_k(s, k, xis, xjs, xj1s, xed, dyds):
    su = (s)**0.5
    orig_shape = xis.shape
    fxis, fxjs, fxj1s, fdyds = xis.reshape(-1), xjs.reshape(-1), xj1s.reshape(-1), dyds.reshape(-1)
    ans = np.zeros_like(fxis)
    inds_0 = np.argwhere(np.isclose(fdyds, 0.)).flatten()
    inds_nnz = np.argwhere(fdyds != 0.).flatten()
    xis_0, xjs_0, xj1s_0 = fxis[inds_0], fxjs[inds_0], fxj1s[inds_0]
    xis_nnz, xjs_nnz, xj1s_nnz = fxis[inds_nnz], fxjs[inds_nnz], fxj1s[inds_nnz]
    if len(inds_0) > 0:
        a1 = su*(xjs_0 + xis_0 - 2*k*xed)
        b1 = su*(xj1s_0 + xis_0 - 2*k*xed)
        a2 = su*(xjs_0 - xis_0 - 2*k*xed)
        b2 = su*(xj1s_0 - xis_0 - 2*k*xed)
        s1 = 1 - 2*(a1 > 0.)
        s2 = 1 - 2*(b1 < 0.)
        s3 = 1 - 2*(a2 > 0.)
        s4 = 1 - 2*(b2 < 0.)
        a1 = np.round(np.abs(a1), decimals=6)
        b1 = np.round(np.abs(b1), decimals=6)
        a2 = np.round(np.abs(a2), decimals=6)
        b2 = np.round(np.abs(b2), decimals=6)
        u1, i1 = np.unique(a1, return_inverse = True)
        u2, i2 = np.unique(b1, return_inverse = True)
        u3, i3 = np.unique(a2, return_inverse = True)
        u4, i4 = np.unique(b2, return_inverse = True)
        ua = np.concatenate((u1, u2, u3, u4))
        u, i = np.unique(ua, return_inverse = True)
        va = 1./su/xed*iti0k0(u)[1][i]
        v1 = va[:len(u1)][i1]
        v2 = va[len(u1):len(u1)+len(u2)][i2]
        v3 = va[len(u1)+len(u2):len(u1)+len(u2)+len(u3)][i3]
        v4 = va[len(u1)+len(u2)+len(u3):][i4]
        ans[inds_0] = v1*s1 + v2*s2 + v3*s3 + v4*s4
    if len(inds_nnz) > 0:
        raise NotImplementedError
    return ans.reshape(orig_shape)

def ifb3_1(s, xis, xjs, xj1s, xed, dyds, debug=False):
    KMAX = 100
    EPS = 1e-12
    TINY = 1e-20
    sum_ = ifb3_k(s, 0, xis, xjs, xj1s, xed, dyds)
    for k in range(1, KMAX):
        d = ifb3_k(s, k, xis, xjs, xj1s, xed, dyds)
        d += ifb3_k(s, -k, xis, xjs, xj1s, xed, dyds)
        sum_ += d
        if np.max(d)/(np.min(sum_) + TINY) < EPS:
            if debug:
                return (0.5*sum_, k)
            else:
                return 0.5*sum_
    raise RuntimeWarning("ifb3_1 did not converge in {} steps".format(KMAX))
    
def ifb3_2(s, xjs, xj1s, xed, dyds):
    su = s**0.5
    return -0.5*np.pi/xed/su*(xj1s - xjs)*np.exp(-su*dyds)

def ifb3(s, xis, xjs, xj1s, xed, dyds):
    return ifb3_1(s, xis, xjs, xj1s, xed, dyds) + ifb3_2(s, xjs, xj1s, xed, dyds)

In [4]:
xis = well.gk["mxd"]
xjs = well.gk["mxj"]
xj1s = well.gk["mxj1"]
dyds = well.gk["myd"]

In [5]:
ans = ifb3_k(s, 0, xis, xjs, xj1s, xed, dyds)

In [12]:
ifb3_1(s, xis, xjs, xj1s, xed, dyds, debug=False)

array([[  1.91676011e-02,   1.09920270e-02,   7.49839964e-03, ...,
          2.07411571e-05,   1.79410540e-05,   1.55243510e-05],
       [  1.09920270e-02,   1.91674346e-02,   1.09918825e-02, ...,
          2.39857296e-05,   2.07409237e-05,   1.79410540e-05],
       [  7.49839964e-03,   1.09918825e-02,   1.91673091e-02, ...,
          2.77484452e-05,   2.39857296e-05,   2.07411571e-05],
       ..., 
       [  2.07411571e-05,   2.39857296e-05,   2.77484452e-05, ...,
          1.91673091e-02,   1.09918825e-02,   7.49839964e-03],
       [  1.79410540e-05,   2.07409237e-05,   2.39857296e-05, ...,
          1.09918825e-02,   1.91674346e-02,   1.09920270e-02],
       [  1.55243510e-05,   1.79410540e-05,   2.07411571e-05, ...,
          7.49839964e-03,   1.09920270e-02,   1.91676011e-02]])

In [13]:
well.source.ifb3_1(s, xis, xjs, xj1s, xed, dyds)

array([[  1.91676011e-02,   1.09920270e-02,   7.49839964e-03, ...,
          2.07411571e-05,   1.79410540e-05,   1.55243510e-05],
       [  1.09920270e-02,   1.91674346e-02,   1.09918825e-02, ...,
          2.39857296e-05,   2.07409237e-05,   1.79410540e-05],
       [  7.49839964e-03,   1.09918825e-02,   1.91673091e-02, ...,
          2.77484452e-05,   2.39857296e-05,   2.07411571e-05],
       ..., 
       [  2.07411571e-05,   2.39857296e-05,   2.77484452e-05, ...,
          1.91673091e-02,   1.09918825e-02,   7.49839964e-03],
       [  1.79410540e-05,   2.07409237e-05,   2.39857296e-05, ...,
          1.09918825e-02,   1.91674346e-02,   1.09920270e-02],
       [  1.55243510e-05,   1.79410540e-05,   2.07411571e-05, ...,
          7.49839964e-03,   1.09920270e-02,   1.91676011e-02]])